In [1]:
import time
import pickle
import scipy
import glob
import sys
import os

import zstandard as zstd
import numpy as np

from scipy.sparse import dok_matrix

scriptpath = "../"
sys.path.append(os.path.abspath(scriptpath))
from helpers.helpers import *

In [2]:
# Set of duplicate users
duplicate_users = dict_occurent_users()

#### Helpers dictionarries

In [3]:
# Dictionnary mapping the video_id to the channel_id
vid_to_channels = video_id_to_channel_id()

In [3]:
# Channels that are in set_crawler dataset and also in which the language is in english
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [5]:
# Dictionnary mapping the channel index to the total number of comments in this channel
with open("/dlabdata1/youtube_large/jouven/comments_per_channel_id.pkl", 'rb') as f:
    channels_idx_to_nb_comments = pickle.load(f)
f.close()

#### Create the bipartite graph between the users and the channels using sparse matrices

In [ ]:
def add_channels_user_tuple(graph_dict, user_channels_occurences):
    graph_dict.update(user_channels_occurences)

            
def update_user_channel_occurences(user_channels_occurences, channel_idx, user_idx):
    if (channel_idx, user_idx) in user_channels_occurences:
        user_channels_occurences[(channel_idx, user_idx)] += 1
    else:
        user_channels_occurences[(channel_idx, user_idx)] = 1

In [ ]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=16384)

# PARAMETERS

# Dictionnary containing channel, user tuple
graph_dict = {}
# Dictionarray counting the numbers of comments of a given user on channels
user_channels_occurences = {}
# Dictionnary giving to a user his corresponding index
user_id_mapping = {}

user = ''
begin_time = time.time()
# Numbers of channels
nb_channels = len(channels_id)
# Numbers of users
nb_users = 406925230
# Users idx
user_idx = -1
# Indices used for prints
idx = 1
nb = 1
nb_user = 1

# Create the training file where each line corresponds to a (word, context) = (channel, user) pairs for word2vecf
f = open("/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/training_data", "w")
f.close()
train = ""
train_temp = ""
training_idx = 1

dir_1 = '/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_for_word2vec'
check_directory(dir_1)

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        author_id = line_split[0]
        if vid_to_channels.get(line_split[2]) in channels_id:
            channel_idx = dict_channel_ind[vid_to_channels[line_split[2]]]
            
            if author_id == user:
                if author_id in duplicate_users:
                    if duplicate_users[author_id] <= 1:
                        update_user_channel_occurences(user_channels_occurences, channel_idx, user_idx)
                        train_temp += str(channel_idx) + " " + str(user_idx) + "\n"
                        training_idx += 1
                else:
                    update_user_channel_occurences(user_channels_occurences, channel_idx, user_idx)
                    train_temp += str(channel_idx) + " " + str(user_idx) + "\n"
                    training_idx += 1
            else:
                if len(user_channels_occurences) > 0:
                    add_channels_user_tuple(graph_dict, user_channels_occurences)
                
                user_channels_occurences = {}
                
                if author_id in duplicate_users:
                    duplicate_users[author_id] += 1
                    if duplicate_users[author_id] <= 1:
                        user_idx += 1
                        update_user_channel_occurences(user_channels_occurences, channel_idx, user_idx)
                        user_id_mapping[author_id] = user_idx
                        train_temp += str(channel_idx) + " " + str(user_idx) + "\n"
                        training_idx += 1
                        
                else:
                    user_idx += 1
                    update_user_channel_occurences(user_channels_occurences, channel_idx, user_idx)
                    user_id_mapping[author_id] = user_idx
                    train_temp += str(channel_idx) + " " + str(user_idx) + "\n"
                    training_idx += 1
                    
    if len(user_id_mapping) % 25000000 == 0 and not len(user_id_mapping) == 0:
        with open("/dlabdata1/youtube_large/jouven/users_index_mapping_comments_more_10k/user_mapping_" + str(nb_user) +".pkl",'wb') as f:
             pickle.dump(user_id_mapping, f)
        f.close()
        nb_user += 1
        user_id_mapping = {}
        
    if training_idx % 10000 == 0:
        # For speed purpose
        train = train + train_temp
        train_temp = ""
        
    if training_idx % 100000000 == 0:
        f = open("/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/training_data", "a")
        f.write(train)
        f.close()
        train = ""
        train_temp = ""
                
    if idx % 75000000 == 0:
        # For space requirements every 75 millions line create a dok matrix and
        # update it with the graph_dict dictionnary and then save it into csr format and then release memory
        graph_matrix = dok_matrix((nb_channels, nb_users), np.uint32)
        dict.update(graph_matrix, graph_dict)
        print('len(graph_dict) ', len(graph_dict))
        print(graph_matrix.count_nonzero())
        print(graph_matrix.tocsr().count_nonzero())
        graph_dict = {}
        # Save sparse matrix
        scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_for_word2vec/matrice' + str(nb) + '.npz', graph_matrix.tocsr())
        with open("/dlabdata1/youtube_large/jouven/sparse_matrix_construction/idx_bip.pkl",'wb') as f:
             pickle.dump([idx], f)
        f.close()
        graph_matrix = []
        nb += 1
        print('line number: ' + str(idx) + ' time: ' + str(time.time() - begin_time))
        begin_time = time.time()
        
    user = line_split[0]
    idx += 1
    

# Store graph
graph_matrix = dok_matrix((nb_channels, nb_users), np.uint32)
dict.update(graph_matrix, graph_dict)
graph_dict = {}
scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_for_word2vec/matrice' + str(nb) + '.npz', graph_matrix.tocsc())
graph_matrix = []

# Store user mapping
with open("/dlabdata1/youtube_large/jouven/users_index_mapping_comments_more_10k/user_mapping_" + str(nb_user) +".pkl",'wb') as f:
     pickle.dump(user_id_mapping, f)
f.close()
user_id_mapping = {}

# Store (channel, user) data into the training file
f = open("/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/training_data", "a")
f.write(train)
f.close()

len(graph_dict)  20518092
20518092
20518092
line number: 75000000 time: 389.45528507232666
len(graph_dict)  20638094
20638094
20638094
line number: 150000000 time: 399.13372325897217
len(graph_dict)  20510391
20510391
20510391
line number: 225000000 time: 396.30357456207275
len(graph_dict)  20456496
20456496
20456496
line number: 300000000 time: 393.6120083332062
len(graph_dict)  20528694
20528694
20528694
line number: 375000000 time: 397.62460684776306
len(graph_dict)  20584053
20584053
20584053
line number: 450000000 time: 398.2904932498932
len(graph_dict)  20564159
20564159
20564159
line number: 525000000 time: 408.5987150669098
len(graph_dict)  20444731
20444731
20444731
line number: 600000000 time: 400.1595838069916
len(graph_dict)  20602555
20602555
20602555
line number: 675000000 time: 402.271853685379
len(graph_dict)  20404779
20404779
20404779
line number: 750000000 time: 409.7469153404236
len(graph_dict)  20569439
20569439
20569439
line number: 825000000 time: 419.46616601943

In [6]:
# Numbers of channels
nb_channels = len(channels_id)
# Numbers of users
nb_users = 406925230

graph = dok_matrix((nb_channels, nb_users), dtype = np.uint32).tocsr()
path = '/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_for_word2vec/'
files = glob.glob(path + '*.npz')
for file in files: 
    graph += scipy.sparse.load_npz(file).astype(np.uint32)

In [7]:
dir_2 = '/dlabdata1/youtube_large/jouven/final_sparse_matrix/'
check_directory(dir_2)
# Save the final sparse matrix
scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/final_sparse_matrix/sparse_matrix_for_word2vec.npz', graph)